# Getting Started
> First steps towards building an IR-based QA system with BERT.

- toc: true 
- badges: true
- comments: true
- hide: true
- permalink: /hidden/
- search_exclude: false
- categories: [jupyter]

## So you've decided to build a QA system. 
You want to start with something general and straightforward so you plan to make it open domain using Wikipedia as a corpus for answering questions. You're going to use an IR-based design (see previous post) since you're working with a large collection of unstructured text. You want to use the best NLP that your compute resources allow (you're lucky enough to have access to a GPU) so you're going to focus on the big, flashy Transformer models that are all the rage these days. 

Sounds like a plan! So where do you start? 

This was our thought process when we first set out on this research path and in this post we'll discuss what you need to know to get going!

- installing libraries and setting up an environment
- understanding Huggingface's `run_squad.py` training script
- Understanding the basic ins and outs of a BERT-esque model
- getting BERT to accept a full Wikipedia article as context for a question


In [4]:
from qasystem import DocumentReader, MODEL_PATHS

In [5]:
reader = DocumentReader(MODEL_PATHS['bert_base_uncased_squad1'])

OSError: Can't load '/home/ryan/work/ff14/src/models/squad1_output/'. Make sure that:

- '/home/ryan/work/ff14/src/models/squad1_output/' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/home/ryan/work/ff14/src/models/squad1_output/' is the correct path to a directory containing a 'config.json' file



In [ ]:
reader